In [4]:
##### DEPENDENCIES #####
!pip install bs4
!pip install zyte
!pip install requests
!pip install pandas
import os

from telnetlib import X3PAD
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
from itertools import chain



You should consider upgrading via the 'C:\Users\brian\OneDrive\Desktop\trust_pilot_scraper\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\brian\OneDrive\Desktop\trust_pilot_scraper\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\brian\OneDrive\Desktop\trust_pilot_scraper\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\brian\OneDrive\Desktop\trust_pilot_scraper\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
###############################  SETTINGS  ########################################
#Tested on v 3.9.7 VENV
INPUT = "input_file.csv"
PAGE_LENGTH_SETTING = 20# Number of pages per brand to scrape comment "ALL" to scrape all pages
NUMBER_OF_BRANDS_TO_SCRAPE = "ALL" #How many brands to scrape on the current list. comment "ALL" to scrape all brand in list
REMOVE_ROWS = 25 #Set the number of rows to remove from the DF. ie. REMOVE_ROWS = 10 -Removes first ten Rows of df
MIN_WORDS = "25"
###################################################################################

In [6]:

result = requests.get(
    "https://httpbin.org/get",
    proxies={
        "http": "http://455fe7d72d1c433b99cb2cf35bb839d4:@proxy.crawlera.com:8011/",
        "https": "http://455fe7d72d1c433b99cb2cf35bb839d4:@proxy.crawlera.com:8011/",
    },
    verify="zyte-smartproxy-ca.crt" 
)


df1 = pd.read_csv(INPUT,engine = "python", error_bad_lines= False )
dataframes = []


attributes_to_remove = ["class","id","style","rel","data-id","target","aria-label","data-type","title",'"col-lg-8 page-content"',"col-lg-8 page-content","alt"] # You can also add id,style,etc in the list

def cleanup(a):
        x_all= [x for x in a if x != "["]
        x_all = [x for x in x_all if x != "]"]
        x_all = [x for x in x_all if x != ", "]
        return x_all
    

if NUMBER_OF_BRANDS_TO_SCRAPE != "ALL":
    df1 =df1.head(NUMBER_OF_BRANDS_TO_SCRAPE)

if REMOVE_ROWS != None:
      df1 =df1.tail(df1.shape[0] -REMOVE_ROWS)

url_list = df1.values.tolist()

for i in url_list :
    URL = i[0]
    rx = requests.get(URL)
    soupx = BeautifulSoup(rx.text, 'html.parser')
    soupx = soupx.find("a", {"name":"pagination-button-last"})

  
    for tag in soupx :
            for attribute in attributes_to_remove: # 
                tag.attrs = {}
            for a in soupx ("a"):
                a.replaceWithChildren()  
    ###print(soup_object_last_div)
    if PAGE_LENGTH_SETTING != "ALL":
        total_pages = PAGE_LENGTH_SETTING
    else:
        total_pages = "".join([str(tag) for tag in soupx])
        #"".join([str(tag) for tag in soupx])
    ###print(soup_object_length_last)
    ##print(total_pages)
    

    for i in range(total_pages):
        
        URL2 = f"{URL}?page={i}"
        if i == 0:
            continue
        
        r = requests.get(URL2)
        #print(f"TEST: {URL2}")

        soup = BeautifulSoup(r.text, 'html.parser')
        #results = soup.find_all(["h1","h2","h3","p","ul"])
        ##print(soup_object2)
        soup_object_title = BeautifulSoup(str(soup.find_all(class_=['title_displayName__TtDDM'])))
        for tag in soup_object_title():
            for attribute in attributes_to_remove : # You can also add id,style,etc in the list
                tag.attrs = {}
            for span in soup_object_title("span"):
                span.replaceWithChildren()
            ###print(str(tag))  

        x_title = [str(tag) for tag in soup_object_title]
        x_title = cleanup(x_title)

     

        soup_object_review_body = BeautifulSoup(str(soup.find_all(class_=['styles_reviewContent__0Q2Tg'])))


        for tag in soup_object_review_body():
            for attribute in attributes_to_remove: # You can also add id,style,etc in the list
                tag.attrs = {}
            for span in soup_object_review_body("span"):
                span.replaceWithChildren()
            
            ###print(str(tag))  

        x_body= [str(tag) for tag in soup_object_review_body]
        x_body = cleanup(x_body)
        
        html_list = []
        date_list = []
        review_length_list = []

        #x_body = x_body.replace("><",">$$$<").split("$$$")
        for i in x_body:
            x = i.replace("<p><b>Date of experience","$$$<p><b>Date of experience")
            i = x.split("$$$")[0]
            len_review = len(i.split(" "))
            review_length_list.append(len_review)
            try:
                date = x.split("$$$")[1]
            except:
                date = ""

            date = date.replace("</p></div>","").replace("<b>Date of experience<!-- -->:</b> <!-- -->","").replace("<b>Date of experience<!-- -->:</b> <!-- -->","").replace("<p>","") #</p></div>
            
            x = i.replace("><",">$$$<").split("$$$")
            array_1 = []
            for item in x:
                item = item.replace("<br/>","")
                if "h2" in item or "<a>" in item or "<div>" in item or "</div>" in item:
                    x=""
                else:
                    item = item.replace("<p>","").replace("</p>","")
                    array_1.append(item)
            remainder_joined = "".join(array_1)        
            html_list.append(remainder_joined)
            date_list.append(date)      

           ##print(html_list)  ###EnD HERE ########################################################### BODY:S
        url_list = []
        list_count = len(html_list)
        x_title =x_title[0]
        x_title_list = [] 
        
        for i in range(list_count):
            x_title_list.append(x_title)
            url_list.append(URL2)
            

        len_x = len(x_title_list)        
        #print(f"X_TITLE: {x_title_list} + Len: {len_x }"
        # 
        # 
        # 
        # 
        # 
        # )
        TITLES = x_title_list ############################################################## TITLES
        #sys.exit()
    

        soup_object4 = BeautifulSoup(str(soup.find_all(class_=['star-rating_starRating__4rrcf'])))
        attr_whitelist = "alt"

        for tag in soup_object4():
            for k in list(tag.attrs.keys()):
                ###print(k)
                if k not in attr_whitelist:
                    tag.attrs.pop(k, None)
            for div in soup_object4("div"):
                div.replaceWithChildren()
            #for p in soup_object4("p"):

        x3 = [str(tag) for tag in soup_object4]
        x3 = cleanup(x3)

        newlist = []
        counter = 1
        for i in x3:

            i = str(i).replace('<img alt="',"").replace('"/>',"").replace('"/>',"")
            if "TrustScore" in i:
                continue
            if "Rated" not in i:
                continue
            i = i.replace(" out of 5 stars","").replace("Rated ","")
            print(i)
            newlist.append(i)
            ###print(counter)
            counter+=1
            ###print(i)


        soup_object2 = BeautifulSoup(str(soup.find_all(class_=['styles_consumerName__dP8Um'])))    
        for tag in soup_object2():
            for attribute in attributes_to_remove: # You can also add id,style,etc in the list
                tag.attrs = {}
            for div in soup_object2("div"):
                div.replaceWithChildren()
            ##print(str(tag))  

        x = [str(tag) for tag in soup_object2]
        x = cleanup(x)
        
        #__next > div > div > main > div > div.styles_mainContent__nFxAv > section > div:nth-child(11) > article > aside > div > a > div.styles_consumerExtraDetails__fxS4S > div > span
        #.styles_consumerInfoWrapper__KP3Ra .styles_consumerDetailsWrapper__p2wdr .styles_consumerExtraDetails__fxS4S .styles_detailsIcon__Fo_ua>*+*
        soup_object_location = BeautifulSoup(str(soup.find_all(class_=['typography_typography__QgicV typography_bodysmall__irytL typography_color-gray-6__TogX2 typography_weight-regular__TWEnf typography_fontstyle-normal__kHyN3 styles_detailsIcon__Fo_ua'])))
        #soup_object_location = BeautifulSoup(str(soup_object_location.find_all("span")))
        
        
        for tag in soup_object_location():
            for attribute in attributes_to_remove: # You can also add id,style,etc in the list
                tag.attrs = {}
            for div in soup_object_location("div"):
                div.replaceWithChildren()
            for svg in soup_object_location("svg"):
                svg.decompose()  
            for path in soup_object_location("path"):
                path.decompose()  
            for g in soup_object_location("g"):
                g.decompose()
            for span in soup_object_location("span"):
                span.replaceWithChildren()
            #for span in soup_object_location("span"):
        #        #span.replaceWithChildren()         
          
        x_loc = [str(tag) for tag in soup_object_location]
        x_loc = cleanup(x_loc)
        
        
        ##print(x)   
        NAME = x    
        REVIEW_BODY = html_list    
        TITLES = x_title_list ############################################################## TITLES    
        STAR_RATINGS = newlist ############################################################## STAR RATINGS
    
        name = pd.DataFrame(NAME,
                         columns = ['Name'])
        review = pd.DataFrame(REVIEW_BODY,
                         columns = ['Review'])
        title= pd.DataFrame(TITLES,
                         columns = ['Title'])
        star_rating = pd.DataFrame(STAR_RATINGS,
                         columns = ['Star Rating'])
        date = pd.DataFrame(date_list,
                         columns = ['Date'])
        review_word_count= pd.DataFrame(review_length_list,
                         columns = ['Review_Wordcount'])
        
        location = pd.DataFrame(x_loc,
                         columns = ['Location'])
        url  = pd.DataFrame(url_list,
                         columns = ['URL'])

        test1 = pd.concat([title,name,review,review_word_count,star_rating,date,location,url], axis=1, join="inner")
        
        try:
            date = x.split("$$$")[1]
        except:
            date = ""
        
        output_path='output.csv'
        
        df1 = test1[test1.Review != '']
        
        if MIN_WORDS != None:
            df1 = df1[df1.Review_Wordcount > int(MIN_WORDS)]
        df1.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
        #dataframes.append(test1)
        ##print(URL2)


#df_all = pd.concat(dataframes)
#df1 = df_all[df_all.Review != '']
#if MIN_WORDS != None:
    #df1 = df_all[df_all.Review_Wordcount > MIN_WORDS]
#df1.to_csv("z_test_1.csv")



C:\Users\brian\AppData\Local\Temp\ipykernel_50940\159162016.py:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv(INPUT,engine = "python", error_bad_lines= False )


4
5
4
5
5
5
5
5
5
5
5
5
1
5
5
5
5
5
5
1
5
5
5
5
1
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
2
5
5
5
5
5
5
5
5
5
5
5
1
5
5
4
5
5
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
5
3
5
5
5
5
5
5
5
4
5
5
5
5
5
5
3
5
1
5
5
5
5
1
5
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
5
5
5
5
5
1
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
4
5
1
5
5
5
5
3
5
5
5
5
5
5
5
5
5
5
5
5
1
5
5
5
5
5
5
3
5
5
5
5
5
3
5
5
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
5
5
5
5
5
4
5
5
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
1
5
5
5
5
5
5
5
5
1
4
5
5
5
5
5
5
5
5
5
5
5
5
1
5
5
5
5
1
1
1
5
5
5
5
1
5
4
5
5
5
5
5
5
1
5
5
5
5
5
5
5
5
1
5
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
5
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
2
5
1
1
1
4
5
5


KeyboardInterrupt: 